In [1]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL database credentials
username = 'root'
password = 'A.11.job.$'
host = 'localhost'
port = '3306'
database = 'sakila'

# Create a connection string
connection_str = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

# Create a SQLAlchemy engine
engine = create_engine(connection_str)



In [2]:
# Query data from MySQL into a Pandas DataFrame
query = 'desc customer'
df = pd.read_sql(query, engine)

# Display the DataFrame
df.head(10)

,Field,Type,Null,Key,Default,Extra
0,customer_id,smallint unsigned,NO,PRI,None,auto_increment
1,store_id,tinyint unsigned,NO,MUL,None,
2,first_name,varchar(45),NO,,None,
3,last_name,varchar(45),NO,MUL,None,
4,email,varchar(50),YES,,None,
5,address_id,smallint unsigned,NO,MUL,None,
6,active,tinyint(1),NO,,1,
7,create_date,datetime,NO,,None,
8,last_update,timestamp,YES,,CURRENT_TIMESTAMP,DEFAULT_GENERATED on update CURRENT_TIMESTAMP


### The Union Operator

In [3]:
query = """
SELECT 'CUST' typ, c.first_name,c.last_name 
FROM customer c
UNION ALL
SELECT 'ACTR' typ, a.first_name, a.last_name
FROM actor a;
"""

In [4]:
df = pd.read_sql(query, engine)

In [5]:
df

,typ,first_name,last_name
0,CUST,MARY,SMITH
1,CUST,PATRICIA,JOHNSON
2,CUST,LINDA,WILLIAMS
3,CUST,BARBARA,JONES
4,CUST,ELIZABETH,BROWN
...,...,...,...
794,ACTR,BELA,WALKEN
795,ACTR,REESE,WEST
796,ACTR,MARY,KEITEL
797,ACTR,JULIA,FAWCETT


##### using %% to prevent ValueError: unsupported format character ''' error

In [6]:
query = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' and c.last_name LIKE 'D%%'
UNION ALL
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' and a.last_name LIKE 'D%%';
"""

In [7]:
df = pd.read_sql(query, engine)

In [8]:
df

,first_name,last_name
0,JENNIFER,DAVIS
1,JENNIFER,DAVIS
2,JUDY,DEAN
3,JODIE,DEGENERES
4,JULIANNE,DENCH


In [9]:
query = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' and c.last_name LIKE 'D%%'
UNION 
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' and a.last_name LIKE 'D%%';
"""

In [10]:
df = pd.read_sql(query, engine)

In [11]:
df

,first_name,last_name
0,JENNIFER,DAVIS
1,JUDY,DEAN
2,JODIE,DEGENERES
3,JULIANNE,DENCH


##### The intersect Operator

In [12]:
query = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'D%%' and c.last_name LIKE 'T%%'
INTERSECT
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'D%%' and a.last_name LIKE 'T%%';
"""

In [13]:
df = pd.read_sql(query, engine)

In [14]:
df

,first_name,last_name


##### Checking for actors and customers having the initials J.D

In [15]:
query = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' and c.last_name LIKE 'D%%'
INTERSECT
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' and a.last_name LIKE 'D%%';
"""

In [16]:
df = pd.read_sql(query, engine)

In [17]:
df

,first_name,last_name
0,JENNIFER,DAVIS


##### Except Operator

In [18]:
query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' and a.last_name LIKE 'D%%'
EXCEPT
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' and c.last_name LIKE 'D%%';
"""

In [19]:
df = pd.read_sql(query, engine)

In [20]:
df

,first_name,last_name
0,JUDY,DEAN
1,JODIE,DEGENERES
2,JULIANNE,DENCH


##### Set Operation Rules

*  give the columns in both queries identical column aliases in order to avoid Error Unknown column ...

**Sorting Compound Query Results**

In [21]:
query = """
SELECT c.first_name fname, c.last_name lname
FROM customer c
WHERE c.first_name LIKE 'J%%' and c.last_name LIKE 'D%%'
UNION ALL
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' and a.last_name LIKE 'D%%'
ORDER BY lname, fname;
"""

In [22]:
df = pd.read_sql(query, engine)

In [23]:
df

,fname,lname
0,JENNIFER,DAVIS
1,JENNIFER,DAVIS
2,JUDY,DEAN
3,JODIE,DEGENERES
4,JULIANNE,DENCH


#### Set Operation Precedence

In [24]:
jd_actor_query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' AND a.last_name LIKE 'D%%';
"""

In [25]:
mt_actor_query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'M%%' AND a.last_name LIKE 'T%%';
"""

In [26]:
jd_customer_query = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' AND c.last_name LIKE 'D%%';
"""

In [27]:
df_jd_act = pd.read_sql(jd_actor_query, engine)

In [28]:
df_jd_act

,first_name,last_name
0,JENNIFER,DAVIS
1,JUDY,DEAN
2,JODIE,DEGENERES
3,JULIANNE,DENCH


In [29]:
df_mt_act = pd.read_sql(mt_actor_query, engine)

In [30]:
df_mt_act

,first_name,last_name
0,MARY,TANDY
1,MENA,TEMPLE


In [31]:
df_jd_cust = pd.read_sql(jd_customer_query, engine)

In [32]:
df_jd_cust

,first_name,last_name
0,JENNIFER,DAVIS


In [33]:
query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' AND a.last_name LIKE 'D%%'
UNION ALL
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'M%%' AND a.last_name LIKE 'T%%'
UNION
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' AND c.last_name LIKE 'D%%'
"""

In [34]:
df = pd.read_sql(query, engine)

In [35]:
df

,first_name,last_name
0,JENNIFER,DAVIS
1,JUDY,DEAN
2,JODIE,DEGENERES
3,JULIANNE,DENCH
4,MARY,TANDY
5,MENA,TEMPLE


In [36]:
query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' AND a.last_name LIKE 'D%%'
UNION 
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'M%%' AND a.last_name LIKE 'T%%'
UNION ALL
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' AND c.last_name LIKE 'D%%'
"""

In [37]:
df = pd.read_sql(query, engine)

In [38]:
df

,first_name,last_name
0,JENNIFER,DAVIS
1,JUDY,DEAN
2,JODIE,DEGENERES
3,JULIANNE,DENCH
4,MARY,TANDY
5,MENA,TEMPLE
6,JENNIFER,DAVIS


#### Using parentheses in compound queries

In [39]:
query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'J%%' AND a.last_name LIKE 'D%%'
UNION 
(
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.first_name LIKE 'M%%' AND a.last_name LIKE 'T%%'
UNION ALL
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.first_name LIKE 'J%%' AND c.last_name LIKE 'D%%'
)
"""

In [40]:
df = pd.read_sql(query, engine)

In [41]:
df

,first_name,last_name
0,JENNIFER,DAVIS
1,JUDY,DEAN
2,JODIE,DEGENERES
3,JULIANNE,DENCH
4,MARY,TANDY
5,MENA,TEMPLE


#### Test your Knowledge Exercise 6

- where A = {L M N O P}
        B = {P Q R S T}
- A union B = {L M N O P Q R S T}
- A union all B = {L M N O P P Q R S T}
- A intersect B = {P}
- A except B = {L M N O}

In [42]:
query = """
SELECT a.first_name, a.last_name
FROM actor a
WHERE a.last_name LIKE 'L%%'
UNION ALL
SELECT c.first_name, c.last_name
FROM customer c
WHERE c.last_name LIKE 'L%%'
"""

In [43]:
df = pd.read_sql(query, engine)

In [44]:
df

,first_name,last_name
0,MATTHEW,LEIGH
1,JOHNNY,LOLLOBRIGIDA
2,MISTY,LAMBERT
3,JACOB,LANCE
4,RENEE,LANE
5,HEIDI,LARSON
6,DARYL,LARUE
7,LAURIE,LAWRENCE
8,JEANNE,LAWSON
9,LAWRENCE,LAWTON


In [45]:
query = """
SELECT a.first_name fname, a.last_name lname
FROM actor a
WHERE a.last_name LIKE 'L%%'
UNION ALL
SELECT c.first_name fname, c.last_name lname
FROM customer c
WHERE c.last_name LIKE 'L%%'
ORDER BY lname;
"""

In [46]:
df = pd.read_sql(query, engine)

In [47]:
df

,fname,lname
0,MISTY,LAMBERT
1,JACOB,LANCE
2,RENEE,LANE
3,HEIDI,LARSON
4,DARYL,LARUE
5,LAURIE,LAWRENCE
6,JEANNE,LAWSON
7,LAWRENCE,LAWTON
8,KIMBERLY,LEE
9,MATTHEW,LEIGH
